In [1]:
### Import the data for MNIST, this will be downloaded if
### necessary or copy it over manually into the directory
### MNIST_data`
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Couldn't import dot_parser, loading of dot files will not be possible.


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
### Import the tensorflow module and create a session
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import scipy

### A helper function to randomly initialise weights
def my_weights(n_inputs, n_outputs, name='default_weights'):
  return tf.Variable(tf.truncated_normal([n_inputs, n_outputs],
               stddev=1.0/math.sqrt(float(n_inputs))), name=name)

In [3]:
### x is for the input y_ holds the output
x = tf.placeholder(tf.float32, shape=[None, 784]) # for the input data 28*28 image as a vector

###
n_D = 784
n_1 = 600
n_2 = 200
n_c = 10

### Encoding layer 1.
W_1 = my_weights(784, n_1, name='W_layer1')
b_1 = tf.Variable(tf.zeros([n_1]))

### Encoding layer 2.
W_2 = my_weights(n_1, n_2, name='W_layer2')
b_2 = tf.Variable(tf.zeros([n_2]))

### Decoding layer 1 (from n_2 to n_1)
dW_1 = my_weights(n_2, n_1, name='Decoding_W_layer1')
db_1 = tf.Variable(tf.zeros([n_1]))

### Decoding layer 2 (from n_1 to n_D)
dW_2 = my_weights(n_1, n_D, name='Decoding_W_layer2')
db_2 = tf.Variable(tf.zeros([n_D]))

### Put it all together...
# Inputs to encoding layer 1
enc_layer1 = tf.nn.sigmoid(tf.matmul(x,W_1) + b_1)

# encoding layer1 to encoding layer 2
enc_layer2 = tf.nn.sigmoid(tf.matmul(enc_layer1, W_2) + b_2)

# encoding layer2 to decoding layer 1
dec_layer1 = tf.nn.sigmoid(tf.matmul(enc_layer2, dW_1) + db_1)

# decoding layer1 to decoding layer 2
dec_layer2 = tf.nn.sigmoid(tf.matmul(dec_layer1, dW_2) + db_2)

In [4]:
### The L2 loss
loss = tf.reduce_mean(tf.pow(dec_layer2-x,2))

### The training algorithm, gradient descent based on cross entropy
#train_step = tf.train.RMSPropOptimizer(0.01).minimize(loss)
train_step = tf.train.AdamOptimizer(0.01).minimize(loss)

In [5]:
### Initialise the variables of the graph e.g. x, y_, W, b
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [6]:
### Go through and do 10 iterations of training, using a batch of 100 samples
display_num = 4
display_image = scipy.zeros((28*2,28*display_num))

for i in range(2000):
  # Randomly select a batch of 100 samples
  batch = mnist.train.next_batch(100)
    
  # Take the images and x is equal to them, y_ is equal to the labels
  _, l2_loss, recon_images = sess.run([train_step, loss, dec_layer2], feed_dict={x: batch[0]})

  #
  if i % 400 == 0:
    print('Step %d, with loss %g' % (i, l2_loss))
    for j in range(display_num):
      display_image[0:28,j*28:(j+1)*28] = batch[0][j].reshape([28,28])
      display_image[28:2*28,j*28:(j+1)*28] = recon_images[j].reshape([28,28])
    plt.figure()
    plt.imshow(display_image, 'gray')
    plt.show()    

Step 0, with loss 0.238358
Step 400, with loss 0.0547871
Step 800, with loss 0.0334347
Step 1200, with loss 0.0245571
Step 1600, with loss 0.0183484
